In [81]:
#pip install pycryptodome
#pip install cryptography

In [82]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import utils
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.backends import default_backend
from Crypto.Cipher import PKCS1_OAEP
import time
import secrets
import hashlib
from datetime import datetime

In [83]:
message = "The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks"

In [84]:
# Current time in seconds since the Unix epoch
timestamp = int(time.time())
print(f"Timestamp: {timestamp}")

Timestamp: 1708879084


In [85]:
# Generate a nonce as a hex string
nonce_hex = secrets.token_hex(16)  # Generates a 32-character hexadecimal string
print(f"Nonce (hex): {nonce_hex}")

Nonce (hex): 0a407794ce24d341a1d6b5f0632b48f3


In [86]:
# Concatenate with a delimiter
combined = f"{nonce_hex}:{timestamp}:{message}"
print("Combined string:", combined)

Combined string: 0a407794ce24d341a1d6b5f0632b48f3:1708879084:The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks


In [87]:
# Encode the combined string to bytes, then hash it using SHA-256
hashed = hashlib.sha256(combined.encode()).hexdigest()

print("SHA-256 Hash:", hashed)

SHA-256 Hash: fc7fc2f8b20cceba8ef031805d932ab631e069893f1fa19f7c21e300044b9123


In [88]:
def generate_rsa_keys():
    """
    Generates an RSA private and public key pair.
    """
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

def serialize_rsa_keys(private_key, public_key):
    """
    Serializes the RSA private and public keys to PEM format.
    """
    pem_private_key = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()  # Use BestAvailableEncryption(password) for encrypted serialization
    )
    
    pem_public_key = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    
    return pem_private_key, pem_public_key

def sign_data_with_private_key(private_key, hashed):
    """
    Signs data using the RSA private key.
    """
    signature = private_key.sign(
        hashed,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature


In [89]:
alice_private_key, alice_public_key = generate_rsa_keys()

pem_private_key, pem_public_key = serialize_rsa_keys(alice_private_key, alice_public_key)

signature = sign_data_with_private_key(alice_private_key, hashed.encode())

In [90]:
# Prepare the message for transmission
transmission_data = {
    'message': combined,
    'signature': signature,
    # 'encrypted_message': encrypted_message,  # If you chose to encrypt
}

# Simulate sending the data by outputting it
print(transmission_data)

{'message': '0a407794ce24d341a1d6b5f0632b48f3:1708879084:The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks', 'signature': b"#\x9adg\x12\xdb,9F\x83\xdf\x10\xd1\xd3#\xf5\x9dj\xe7\xef\xff\xc8D\xa3\x8e\xae\xdc\x0c\xaaoz3\xabNM\x02@(\r^\xfdz\xe1Fy\x98\xac\xcb^LT\xcd\xd9\x0e\xec\xbd\xb0\xba'\x1ffk\xe0|~\xb9x\x15\x95\xb5\x85}\xef?\x04\x1e\x9e\xb1\xe1cn\x16]P\x12d\t\x1bbn\xefEg<Q{\xccn3&\x89\xe8\x82\xf5F\xf5iu\x9b\xb3y\x92\xc4\x1a \x0c\xd8\xc5A\x089\xf0;{\xd7\xd3\xdb\xb2\x9c7\x038.\xe0;\xba\xfd\xb0\x9e>\x08\xc1\x84e\x11?.\xfd\xb9\xe6\x9e\xad'<\xb3\xf9\xb0R\x0e:\xbf\x01\xd4\xfa\x1b\x16\xccHa\x93\x08T\xe4G\xaa\xa7\xf5\xe4\x96\x81\xcc\xd0}\xa9\xed\xc3p\xd4\xf7h \x8e\xec\xd2:hl\xe4TQ~0&,[\x01j\xff\xfcI\xc6\xb5_\x03vw\t\x89\x94I(\xf8\x1d\x14\x00\xf6\x8e\x9e\xdf\xe5\xfb\x01\x89L\x9cU\x9aT\xe6\x9eF\xbf\x9d>\x99Av\x

In [91]:
def verify_signature(received_message, signature, public_key):
    """
    Verifies the signature of the received message using the sender's public key.
    """
    try:
        public_key.verify(
            signature,
            received_message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except InvalidSignature:
        return False

In [92]:
# Simulated received data that Bob recieves from Alice
received_data = transmission_data 

# Verify the signature of the received message
received_hashed_message = hashlib.sha256(received_data['message'].encode()).hexdigest()
verification_result = verify_signature(received_hashed_message.encode(), received_data['signature'], alice_public_key)

print(f"Signature verification result: {verification_result}")

Signature verification result: True


In [93]:
received_message = received_data['message']

parts = received_message.split(':')
nonce = parts[0]
timestamp_str = parts[1]
actual_message = ':'.join(parts[2:])  # Join back the rest of the message in case there are semi-colons in it

# Convert the timestamp string to an integer
timestamp = int(timestamp_str)

# Convert the timestamp to a readable date format
dt_object = datetime.fromtimestamp(timestamp)
readable_date = dt_object.strftime('%Y-%m-%d %H:%M:%S')

print(f"Nonce: {nonce}")
print(f"Readable Timestamp: {readable_date}")
print(f"Message: {actual_message}")

Nonce: 0a407794ce24d341a1d6b5f0632b48f3
Readable Timestamp: 2024-02-25 16:38:04
Message: The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks
